# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [66]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [67]:
df = pd.read_csv('cal_housing_clean.csv')

In [68]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [69]:
df.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [70]:
y_values = df.medianHouseValue

In [71]:
x_values = df.drop('medianHouseValue',axis = 1)

In [72]:
from sklearn.model_selection import train_test_split

In [121]:
X_train, X_test ,y_train,y_test = train_test_split(x_values,y_values,test_size=0.3,random_state=101)

In [105]:
x_values.size

123840

In [106]:
y_values.size

20640

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [107]:
from sklearn.preprocessing import MinMaxScaler

In [108]:
scaler = MinMaxScaler() 

In [109]:
scaler.fit(x_values)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [110]:
# reset x_values to be scaled version of x values
X_train =pd.DataFrame(data = scaler.transform(x_values),columns = x_values.columns,index = x_values.index)

In [111]:
X_test = pd.DataFrame(data = scaler.transform(x_values),columns = x_values.columns,index = x_values.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [112]:
df.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [113]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms =tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households =tf.feature_column.numeric_column('households')
income =tf.feature_column.numeric_column('medianIncome')


In [114]:
feat_cols = [age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [131]:
input_func = tf.estimator.inputs.pandas_input_fn(x =X_train,y= y_train,batch_size =10,num_epochs =10000,shuffle =True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [132]:
# 3 hidden layers of 6 neurons (creating neural network)
model = tf.estimator.DNNRegressor(hidden_units = [6,6,6], feature_columns = feat_cols)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\USER\\AppData\\Local\\Temp\\tmp6607xims', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D72ED7AA58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}



##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [140]:
model.train(input_fn=input_func,steps=200000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\USER\AppData\Local\Temp\tmp6607xims\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 20000 into C:\Users\USER\AppData\Local\Temp\tmp6607xims\model.ckpt.
INFO:tensorflow:loss = 226767670000.0, step = 20001
INFO:tensorflow:global_step/sec: 550.33
INFO:tensorflow:loss = 25615673000.0, step = 20101 (0.197 sec)
INFO:tensorflow:global_step/sec: 585.269
INFO:tensorflow:loss = 97069924000.0, step = 20201 (0.155 sec)
INFO:tensorflow:global_step/sec: 573.214
INFO:tensorflow:loss = 292615700000.0, step = 20301 (0.174 sec)
INFO:tensorflow:global_step/sec: 621.192
INFO:tensorflow:loss = 137217100000.0, step = 20401 (0.161 sec)
INFO:tensorflow:global_step/sec: 598.239
INFO:tensorflow:loss = 108684290000.0, ste

INFO:tensorflow:global_step/sec: 547.761
INFO:tensorflow:loss = 141488370000.0, step = 27501 (0.183 sec)
INFO:tensorflow:global_step/sec: 585.802
INFO:tensorflow:loss = 160466010000.0, step = 27601 (0.171 sec)
INFO:tensorflow:global_step/sec: 618.738
INFO:tensorflow:loss = 64906610000.0, step = 27701 (0.161 sec)
INFO:tensorflow:global_step/sec: 655.311
INFO:tensorflow:loss = 115582080000.0, step = 27801 (0.153 sec)
INFO:tensorflow:global_step/sec: 599.662
INFO:tensorflow:loss = 125716726000.0, step = 27901 (0.177 sec)
INFO:tensorflow:global_step/sec: 602.036
INFO:tensorflow:loss = 230218890000.0, step = 28001 (0.155 sec)
INFO:tensorflow:global_step/sec: 620.5
INFO:tensorflow:loss = 127723730000.0, step = 28101 (0.161 sec)
INFO:tensorflow:global_step/sec: 625.532
INFO:tensorflow:loss = 165255700000.0, step = 28201 (0.160 sec)
INFO:tensorflow:global_step/sec: 604.588
INFO:tensorflow:loss = 99108950000.0, step = 28301 (0.165 sec)
INFO:tensorflow:global_step/sec: 654.709
INFO:tensorflow:lo

INFO:tensorflow:global_step/sec: 643.137
INFO:tensorflow:loss = 40295550000.0, step = 35401 (0.155 sec)
INFO:tensorflow:global_step/sec: 603.204
INFO:tensorflow:loss = 80047440000.0, step = 35501 (0.166 sec)
INFO:tensorflow:global_step/sec: 659.322
INFO:tensorflow:loss = 156145430000.0, step = 35601 (0.167 sec)
INFO:tensorflow:global_step/sec: 541.145
INFO:tensorflow:loss = 65660043000.0, step = 35701 (0.169 sec)
INFO:tensorflow:global_step/sec: 657.213
INFO:tensorflow:loss = 106896230000.0, step = 35801 (0.152 sec)
INFO:tensorflow:global_step/sec: 626.924
INFO:tensorflow:loss = 188342530000.0, step = 35901 (0.175 sec)
INFO:tensorflow:global_step/sec: 557.461
INFO:tensorflow:loss = 143198750000.0, step = 36001 (0.165 sec)
INFO:tensorflow:global_step/sec: 615.705
INFO:tensorflow:loss = 58839160000.0, step = 36101 (0.161 sec)
INFO:tensorflow:global_step/sec: 664.413
INFO:tensorflow:loss = 81152240000.0, step = 36201 (0.151 sec)
INFO:tensorflow:global_step/sec: 522.948
INFO:tensorflow:los

INFO:tensorflow:global_step/sec: 548.493
INFO:tensorflow:loss = 43180614000.0, step = 43301 (0.182 sec)
INFO:tensorflow:global_step/sec: 662.67
INFO:tensorflow:loss = 85234704000.0, step = 43401 (0.167 sec)
INFO:tensorflow:global_step/sec: 600.864
INFO:tensorflow:loss = 74955110000.0, step = 43501 (0.166 sec)
INFO:tensorflow:global_step/sec: 596.607
INFO:tensorflow:loss = 57696510000.0, step = 43601 (0.152 sec)
INFO:tensorflow:global_step/sec: 600.966
INFO:tensorflow:loss = 87817904000.0, step = 43701 (0.166 sec)
INFO:tensorflow:global_step/sec: 599.893
INFO:tensorflow:loss = 110683330000.0, step = 43801 (0.167 sec)
INFO:tensorflow:global_step/sec: 657.178
INFO:tensorflow:loss = 61748970000.0, step = 43901 (0.168 sec)
INFO:tensorflow:global_step/sec: 599.497
INFO:tensorflow:loss = 79036910000.0, step = 44001 (0.167 sec)
INFO:tensorflow:global_step/sec: 545.822
INFO:tensorflow:loss = 154453070000.0, step = 44101 (0.168 sec)
INFO:tensorflow:global_step/sec: 658.472
INFO:tensorflow:loss =

INFO:tensorflow:global_step/sec: 606.688
INFO:tensorflow:loss = 84920610000.0, step = 51201 (0.165 sec)
INFO:tensorflow:global_step/sec: 600.605
INFO:tensorflow:loss = 100413374000.0, step = 51301 (0.182 sec)
INFO:tensorflow:global_step/sec: 593.102
INFO:tensorflow:loss = 24537395000.0, step = 51401 (0.153 sec)
INFO:tensorflow:global_step/sec: 604.91
INFO:tensorflow:loss = 102013050000.0, step = 51501 (0.165 sec)
INFO:tensorflow:global_step/sec: 650.926
INFO:tensorflow:loss = 131322870000.0, step = 51601 (0.169 sec)
INFO:tensorflow:global_step/sec: 599.286
INFO:tensorflow:loss = 117530770000.0, step = 51701 (0.173 sec)
INFO:tensorflow:global_step/sec: 557.484
INFO:tensorflow:loss = 80326255000.0, step = 51801 (0.173 sec)
INFO:tensorflow:global_step/sec: 586.893
INFO:tensorflow:loss = 272654480000.0, step = 51901 (0.155 sec)
INFO:tensorflow:global_step/sec: 592.952
INFO:tensorflow:loss = 55945190000.0, step = 52001 (0.169 sec)
INFO:tensorflow:global_step/sec: 574.472
INFO:tensorflow:los

...

INFO:tensorflow:global_step/sec: 547.946
INFO:tensorflow:loss = 211709800000.0, step = 184401 (0.182 sec)
INFO:tensorflow:global_step/sec: 665.262
INFO:tensorflow:loss = 125943960000.0, step = 184501 (0.166 sec)
INFO:tensorflow:global_step/sec: 599.668
INFO:tensorflow:loss = 108356760000.0, step = 184601 (0.151 sec)
INFO:tensorflow:global_step/sec: 584.199
INFO:tensorflow:loss = 60111700000.0, step = 184701 (0.171 sec)
INFO:tensorflow:global_step/sec: 636.397
INFO:tensorflow:loss = 76994710000.0, step = 184801 (0.157 sec)
INFO:tensorflow:global_step/sec: 578.138
INFO:tensorflow:loss = 22533194000.0, step = 184901 (0.173 sec)
INFO:tensorflow:global_step/sec: 653.678
INFO:tensorflow:loss = 44712276000.0, step = 185001 (0.153 sec)
INFO:tensorflow:global_step/sec: 664.996
INFO:tensorflow:loss = 48150080000.0, step = 185101 (0.166 sec)
INFO:tensorflow:global_step/sec: 593.358
INFO:tensorflow:loss = 48440746000.0, step = 185201 (0.169 sec)
INFO:tensorflow:global_step/sec: 600.056
INFO:tensor

INFO:tensorflow:global_step/sec: 596.029
INFO:tensorflow:loss = 71081450000.0, step = 192201 (0.168 sec)
INFO:tensorflow:global_step/sec: 592.135
INFO:tensorflow:loss = 55190274000.0, step = 192301 (0.169 sec)
INFO:tensorflow:global_step/sec: 671.821
INFO:tensorflow:loss = 62776107000.0, step = 192401 (0.164 sec)
INFO:tensorflow:global_step/sec: 600.501
INFO:tensorflow:loss = 94790110000.0, step = 192501 (0.167 sec)
INFO:tensorflow:global_step/sec: 572.292
INFO:tensorflow:loss = 111465220000.0, step = 192601 (0.159 sec)
INFO:tensorflow:global_step/sec: 569.114
INFO:tensorflow:loss = 74057440000.0, step = 192701 (0.176 sec)
INFO:tensorflow:global_step/sec: 657.479
INFO:tensorflow:loss = 89200960000.0, step = 192801 (0.152 sec)
INFO:tensorflow:global_step/sec: 599.022
INFO:tensorflow:loss = 111433230000.0, step = 192901 (0.167 sec)
INFO:tensorflow:global_step/sec: 599.531
INFO:tensorflow:loss = 50460025000.0, step = 193001 (0.167 sec)
INFO:tensorflow:global_step/sec: 595.497
INFO:tensorf

INFO:tensorflow:global_step/sec: 506.635
INFO:tensorflow:loss = 167563170000.0, step = 200001 (0.197 sec)
INFO:tensorflow:global_step/sec: 601.131
INFO:tensorflow:loss = 43976010000.0, step = 200101 (0.166 sec)
INFO:tensorflow:global_step/sec: 540.619
INFO:tensorflow:loss = 79328490000.0, step = 200201 (0.194 sec)
INFO:tensorflow:global_step/sec: 423.069
INFO:tensorflow:loss = 54410630000.0, step = 200301 (0.235 sec)
INFO:tensorflow:global_step/sec: 435.941
INFO:tensorflow:loss = 151948330000.0, step = 200401 (0.225 sec)
INFO:tensorflow:global_step/sec: 459.941
INFO:tensorflow:loss = 48117084000.0, step = 200501 (0.218 sec)
INFO:tensorflow:global_step/sec: 417.783
INFO:tensorflow:loss = 20253039000.0, step = 200601 (0.242 sec)
INFO:tensorflow:global_step/sec: 455.76
INFO:tensorflow:loss = 107284775000.0, step = 200701 (0.217 sec)
INFO:tensorflow:global_step/sec: 410.929
INFO:tensorflow:loss = 156470410000.0, step = 200801 (0.243 sec)
INFO:tensorflow:global_step/sec: 437.85
INFO:tensorf

INFO:tensorflow:loss = 92810760000.0, step = 207801 (0.184 sec)
INFO:tensorflow:global_step/sec: 580.854
INFO:tensorflow:loss = 59507580000.0, step = 207901 (0.174 sec)
INFO:tensorflow:global_step/sec: 609.924
INFO:tensorflow:loss = 32813535000.0, step = 208001 (0.157 sec)
INFO:tensorflow:global_step/sec: 592.978
INFO:tensorflow:loss = 85126360000.0, step = 208101 (0.169 sec)
INFO:tensorflow:global_step/sec: 624.319
INFO:tensorflow:loss = 47986900000.0, step = 208201 (0.160 sec)
INFO:tensorflow:global_step/sec: 609.76
INFO:tensorflow:loss = 47996410000.0, step = 208301 (0.180 sec)
INFO:tensorflow:global_step/sec: 613.375
INFO:tensorflow:loss = 68944510000.0, step = 208401 (0.163 sec)
INFO:tensorflow:global_step/sec: 511.559
INFO:tensorflow:loss = 144535220000.0, step = 208501 (0.182 sec)
INFO:tensorflow:global_step/sec: 613.621
INFO:tensorflow:loss = 82466900000.0, step = 208601 (0.161 sec)
INFO:tensorflow:global_step/sec: 687.965
INFO:tensorflow:loss = 12389853000.0, step = 208701 (0.

INFO:tensorflow:loss = 82884805000.0, step = 215601 (0.189 sec)
INFO:tensorflow:global_step/sec: 524.957
INFO:tensorflow:loss = 30564458000.0, step = 215701 (0.193 sec)
INFO:tensorflow:global_step/sec: 685.718
INFO:tensorflow:loss = 51184110000.0, step = 215801 (0.155 sec)
INFO:tensorflow:global_step/sec: 557.294
INFO:tensorflow:loss = 62011654000.0, step = 215901 (0.164 sec)
INFO:tensorflow:global_step/sec: 660.477
INFO:tensorflow:loss = 19882064000.0, step = 216001 (0.167 sec)
INFO:tensorflow:global_step/sec: 594.685
INFO:tensorflow:loss = 51095300000.0, step = 216101 (0.153 sec)
INFO:tensorflow:global_step/sec: 634.489
INFO:tensorflow:loss = 72829060000.0, step = 216201 (0.158 sec)
INFO:tensorflow:global_step/sec: 622.727
INFO:tensorflow:loss = 132604570000.0, step = 216301 (0.176 sec)
INFO:tensorflow:global_step/sec: 597.9
INFO:tensorflow:loss = 35993280000.0, step = 216401 (0.152 sec)
INFO:tensorflow:global_step/sec: 645.204
INFO:tensorflow:loss = 165183030000.0, step = 216501 (0.

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [141]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size =10,num_epochs = 1,shuffle = False)

In [142]:
pred_gen = model.predict(predict_input_func)

In [143]:
predictions = list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\USER\AppData\Local\Temp\tmp6607xims\model.ckpt-220000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [144]:
final_preds =[]
for pred in predictions:
    final_preds.append(pred['predictions'])

In [145]:
from sklearn.metrics import mean_squared_error

In [146]:
mean_squared_error(y_test,final_preds)**0.5

81545.12913940175

# Great Job!